In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
interval_kinetics.py
~~~~~~~~~~~~~~~~~~~~
Interval-to-interval kinetic analysis for CHO fed-batch cultures
(Clone × Rep), covering the **entire cultivation** timeline.

Workflow
--------
1. Load the cleaned experimental CSV (`data/data.csv`), skipping the first
   metadata row.
2. Parse the boolean flag `is_post_feed` to distinguish *pre-* and *post-*feed
   samples.
3. Iterate through each Clone × Rep dataset and, for every valid interval
   (t₀ → t₁):
   • Compute µ and IVCD  
   • Calculate total balances for viable cells, glucose, lactate, glutamine,
     glutamate and recombinant protein (rP)  
   • Derive biomass yields and specific rates (q_G, q_L, q_Gln, q_Glu, q_rP)
4. Append the new kinetic columns to the original table.
5. Save the enriched DataFrame.

New features (06 Aug 2025)
--------------------------
• Glutamine / glutamate balances, yields and specific rates  
• Recombinant-protein balance (dP) and q_rP (pg·cell⁻¹·h⁻¹)  
• Accepts rP concentration column **`rP_mg_L`** (mg·L⁻¹)

Inputs
------
data/data.csv  
(required columns: Clone, Rep, t_hr, VCD, Vol_mL, Glc_g_L, Lac_g_L,
Gln_mM, Glu_mM, rP_mg_L, is_post_feed)

Outputs
-------
Enriched CSV → `outputs/interval_kinetics.csv`

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 06 Aug 2025 — docstring overhaul & new kinetics
"""

import numpy as np
import pandas as pd
from pathlib import Path

# ───── Configuration ─────────────────────────────────────────────────── #
DATA_FILE = Path("data/data.csv")
OUTFILE   = Path("outputs/interval_kinetics.csv")

MM_GLUCOSE = 180.156   # g·mol⁻¹
MM_LACTATE =  90.080   # g·mol⁻¹
PG_PER_G   = 1e12      # g → pg

KIN_COLS = [
    "mu", "IVCD_tot",
    "dX", "dG", "dL", "dQln", "dQlu", "dP",
    "Y_XG", "Y_XL", "Y_XQln", "Y_XQlu",
    "q_G", "q_L", "q_Gln", "q_Glu", "q_rP"
]

# ───── Load data ─────────────────────────────────────────────────────── #
if not DATA_FILE.exists():
    raise FileNotFoundError(f"❌ Input file not found:\n  {DATA_FILE}")

df = (
    pd.read_csv(DATA_FILE, skiprows=1)
      .assign(
          t_hr  = lambda d: pd.to_numeric(d["t_hr"], errors="coerce"),
          Rep   = lambda d: pd.Categorical(
                     pd.to_numeric(d["Rep"], errors="coerce"),
                     categories=[1, 2, 3], ordered=True),
          Clone = lambda d: d["Clone"].astype("category"),
          Date  = lambda d: pd.to_datetime(d["Date"], format="%d/%m/%Y", errors="coerce"),
          is_post_feed = lambda d: (
              d["is_post_feed"]
                .fillna(False)
                .apply(lambda x: str(x).strip().lower() in {"true","t","1"})
          ),
          # ensure numeric for new analytes
          Gln_mM   = lambda d: pd.to_numeric(d["Gln_mM"],   errors="coerce"),
          Glu_mM   = lambda d: pd.to_numeric(d["Glu_mM"],   errors="coerce"),
          rP_mg_L  = lambda d: pd.to_numeric(d["rP_mg_L"],  errors="coerce")
      )
      .sort_values(["Clone", "Rep", "t_hr"], ignore_index=True)
)

# ───── Unit conversions ──────────────────────────────────────────────── #
df["Glc_mM"]         = df["Glc_g_L"] / MM_GLUCOSE * 1e3
df["Lac_mM"]         = df["Lac_g_L"] / MM_LACTATE * 1e3

df["Glucose_mol_mL"] = df["Glc_mM"] * 1e-6
df["Lactate_mol_mL"] = df["Lac_mM"] * 1e-6
df["Gln_mol_mL"]     = df["Gln_mM"] * 1e-6
df["Glu_mol_mL"]     = df["Glu_mM"] * 1e-6
df["rP_g_mL"]        = df["rP_mg_L"] * 1e-6   # mg·L⁻¹ → g·mL⁻¹ 

df[KIN_COLS] = np.nan

# ───── Kinetic calculations ─────────────────────────────────────────── #
for (clone, rep), g_raw in df.groupby(["Clone","Rep"], observed=True):
    g = g_raw.sort_values("t_hr").reset_index()
    idx_df = g["index"]

    for i in range(1, len(g)):
        t1 = g.loc[i]

        # interval selection logic
        if t1["t_hr"] <= 72:
            t0 = g.loc[i-1]
        elif not t1["is_post_feed"]:
            pre = g[(g["t_hr"] < t1["t_hr"]) & g["is_post_feed"]]
            if pre.empty: continue
            t0 = pre.iloc[-1]
        else:
            continue

        Δt = t1["t_hr"] - t0["t_hr"]
        if Δt <= 0: continue

        # Growth rate
        mu = (np.log(t1["VCD"]) - np.log(t0["VCD"])) / Δt

        # Total balances (g or mol)
        dX   = t1["VCD"]*t1["Vol_mL"] - t0["VCD"]*t0["Vol_mL"]
        dG   = t0["Glucose_mol_mL"]*t0["Vol_mL"] - t1["Glucose_mol_mL"]*t1["Vol_mL"]
        dL   = t1["Lactate_mol_mL"]*t1["Vol_mL"] - t0["Lactate_mol_mL"]*t0["Vol_mL"]
        dQln = t0["Gln_mol_mL"]*t0["Vol_mL"] - t1["Gln_mol_mL"]*t1["Vol_mL"]
        dQlu = t1["Glu_mol_mL"]*t1["Vol_mL"] - t0["Glu_mol_mL"]*t0["Vol_mL"]
        dP   = t1["rP_g_mL"]*t1["Vol_mL"] - t0["rP_g_mL"]*t0["Vol_mL"]

        # Yields
        Y_XG   = dX/dG   if dG   else np.nan
        Y_XL   = dX/dL   if dL   else np.nan
        Y_XQln = dX/dQln if dQln else np.nan
        Y_XQlu = dX/dQlu if dQlu else np.nan

        # Integrated viable cell density
        IVCD_tot = ((t0["VCD"]+t1["VCD"])/2)*Δt * ((t0["Vol_mL"]+t1["Vol_mL"])/2)

        # Specific rates
        q_G   = (dG  *1e12)/IVCD_tot if IVCD_tot else np.nan
        q_L   = (dL  *1e12)/IVCD_tot if IVCD_tot else np.nan
        q_Gln = (dQln*1e12)/IVCD_tot if IVCD_tot else np.nan
        q_Glu = (dQlu*1e12)/IVCD_tot if IVCD_tot else np.nan
        q_rP  = (dP  *PG_PER_G)/IVCD_tot if IVCD_tot else np.nan  # pg·cell⁻¹·h⁻¹

        # write back
        df.loc[idx_df[i], KIN_COLS] = [
            mu, IVCD_tot, dX, dG, dL, dQln, dQlu, dP,
            Y_XG, Y_XL, Y_XQln, Y_XQlu,
            q_G, q_L, q_Gln, q_Glu, q_rP
        ]

# ───── Save and summary ─────────────────────────────────────────────── #
OUTFILE.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTFILE, index=False)

if __name__ == "__main__":
    print(f"✓ Intervals analysed: {df['mu'].notna().sum()}")
    print(f"✓ File saved → {OUTFILE}")

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
plot_raw.py
~~~~~~~~~~~
Per-sample scatter plots (Clone × Rep) for CHO fed-batch datasets.

Workflow
--------
1. Load the interval-enriched file produced by `interval_kinetics.py`
   (`outputs/interval_kinetics.csv`).
2. Build three plot families:
   • *Time-course* – raw variables vs. sampling time (t_hr)  
   • *Kinetics*   – derived balances / rates / yields vs. time  
   • *Correlations* – scatter plots between selected metric pairs
     (one point per sample, coloured by Clone and shaped by Rep)
3. Save every figure as a high-resolution PNG inside a structured folder tree.

Inputs
------
outputs/interval_kinetics.csv  
(required columns listed in the code: Clone, Rep, t_hr, and all kinetic
columns generated on 06 Aug 2025 or later, including `rP_mg_L`.)

Outputs
-------
PNG files in `outputs/figures_raw/`
├── time/      # raw variables  
├── kinetics/  # derived metrics  
└── corr/      # correlations

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 06 Aug 2025 — rP_mg_L nomenclature & safe legend
"""


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# ───── Configuration ─────────────────────────────────────────────────── #
CSV_PATH   = Path("outputs/interval_kinetics.csv")
FIGURE_DIR = Path("outputs/figures_raw")
SUBFOLDERS = ["time", "kinetics", "corr"]
FIGSIZE, DPI = (8, 6), 300
AXES_RECT  = [0.15, 0.15, 0.78, 0.78]
PALETTE    = "tab10"
RATE_SET   = {"mu", "q_G", "q_L", "q_Gln", "q_Glu", "q_rP"}  # rates → ejes ≥ 0

sns.set_style("whitegrid")
SHAPE_MAP = {1: "o", 2: "s", 3: "D"}

# ───── Helpers ───────────────────────────────────────────────────────── #
def scatter_by_rep(ax, data, x, y):
    for cl, g_cl in data.groupby("Clone", observed=True, sort=False):
        for rp, g_rp in g_cl.groupby("Rep", observed=True, sort=False):
            ax.scatter(g_rp[x], g_rp[y],
                       color=COLOR[cl],
                       marker=SHAPE_MAP.get(rp, "o"),
                       s=65, edgecolor="white", linewidth=0.4,
                       label=f"{cl}-rep{rp}")

def safe_legend(ax, **kwargs):
    h, l = ax.get_legend_handles_labels()
    if h: ax.legend(**kwargs)

# ───── Load data & folders ───────────────────────────────────────────── #
if not CSV_PATH.exists():
    raise FileNotFoundError(f"❌ File not found:\n  {CSV_PATH}")
df = pd.read_csv(CSV_PATH)

for sub in SUBFOLDERS:
    (FIGURE_DIR / sub).mkdir(parents=True, exist_ok=True)

clones = df["Clone"].unique().tolist()
colors = sns.color_palette(PALETTE, len(clones))
COLOR  = dict(zip(clones, colors))

# ───── 1. Raw time-course ────────────────────────────────────────────── #
PLOT_TIME = [
    ("VCD",       r'VCD (cells·mL$^{-1}$)',          "Viable Cell Density"),
    ("Viab_pct",  r'Viability (%)',                  "Cell Viability"),
    ("Glc_mM",    r'Glucose (mM)',                   "Glucose Concentration"),
    ("Lac_mM",    r'Lactate (mM)',                   "Lactate Concentration"),
    ("Gln_mM",    r'Glutamine (mM)',                 "Glutamine Concentration"),
    ("Glu_mM",    r'Glutamate (mM)',                 "Glutamate Concentration"),
    ("rP_mg_L",   r'Protein (mg·L$^{-1}$)',          "Recombinant Protein"),  # ← updated
    ("GFP_mean",  r'GFP (a.u.)',                     "GFP Mean Fluorescence"),
    ("TMRM_mean", r'TMRM (a.u.)',                    "TMRM Mean Fluorescence"),
]

for var, ylab, title in PLOT_TIME:
    if var not in df.columns:
        print(f"⚠️  '{var}' not found; skipping.")
        continue
    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax  = fig.add_axes(AXES_RECT)
    scatter_by_rep(ax, df, "t_hr", var)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel(ylab)
    ax.set_title(title)
    ax.set_xlim(left=0)
    safe_legend(ax, title="Clone–Rep", fontsize=8)
    fig.savefig(FIGURE_DIR / "time" / f"{var}_raw.png")
    plt.close(fig)

print("✓ Time trends saved")

# ───── 2. Kinetic parameters ─────────────────────────────────────────── #
PLOT_KIN = [
    ("mu",        r'μ (h$^{-1}$)',                       "Specific Growth Rate"),
    ("IVCD_tot",  r'IVCD (cells·h)',                     "Integral Viable Cell Density"),
    ("dX",        r'ΔX (cells)',                         "Net Cell Change"),
    ("dG",        r'ΔGlucose (mol)',                     "Net Glucose Consumption"),
    ("dL",        r'ΔLactate (mol)',                     "Net Lactate Production"),
    ("dQln",      r'ΔGlutamine (mol)',                   "Net Glutamine Consumption"),
    ("dQlu",      r'ΔGlutamate (mol)',                   "Net Glutamate Production"),
    ("dP",        r'ΔProtein (g)',                       "Net rP Accumulation"),
    ("q_G",       r'q$_G$ (pmol·cell$^{-1}$·h$^{-1}$)',  "Specific Glucose Consumption"),
    ("q_L",       r'q$_L$ (pmol·cell$^{-1}$·h$^{-1}$)',  "Specific Lactate Production"),
    ("q_Gln",     r'q$_{Gln}$ (pmol·cell$^{-1}$·h$^{-1}$)',"Specific Glutamine Consumption"),
    ("q_Glu",     r'q$_{Glu}$ (pmol·cell$^{-1}$·h$^{-1}$)',"Specific Glutamate Production"),
    ("q_rP",      r'q$_{rP}$ (pg·cell$^{-1}$·h$^{-1}$)', "Specific rP Production"),
    ("Y_XG",      r'Y$_{X/G}$ (cells·mol$^{-1}$)',       "Yield on Glucose"),
    ("Y_XL",      r'Y$_{X/L}$ (cells·mol$^{-1}$)',       "Yield on Lactate"),
    ("Y_XQln",    r'Y$_{X/Gln}$ (cells·mol$^{-1}$)',     "Yield on Glutamine"),
    ("Y_XQlu",    r'Y$_{X/Glu}$ (cells·mol$^{-1}$)',     "Yield on Glutamate"),
]

for var, ylab, title in PLOT_KIN:
    if var not in df.columns:
        continue

    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)     
    ax  = fig.add_axes(AXES_RECT)                  

    scatter_by_rep(ax, df, "t_hr", var)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel(ylab)
    ax.set_title(title)
    ax.set_xlim(left=0)
    safe_legend(ax, title="Clone–Rep", fontsize=8)

    fig.savefig(FIGURE_DIR / "kinetics" / f"{var}_raw.png")
    plt.close(fig)

print("✓ Kinetic plots saved")

# ───── 3. Correlations ───────────────────────────────────────────────── #
PAIR_CORR = [
    ("mu",  "q_G"), ("mu",  "q_L"), ("mu",  "q_Gln"),
    ("mu",  "q_Glu"), ("mu",  "q_rP"),
    ("q_G", "q_L"),  ("q_Gln", "q_Glu"), ("q_G", "q_rP"),
    ("mu",  "GFP_mean"), ("mu", "TMRM_mean"),
]

LABELS = {
    "mu":       r'μ (h$^{-1}$)',
    "q_G":      r'q$_G$ (pmol·cell$^{-1}$·h$^{-1}$)',
    "q_L":      r'q$_L$ (pmol·cell$^{-1}$·h$^{-1}$)',
    "q_Gln":    r'q$_{Gln}$ (pmol·cell$^{-1}$·h$^{-1}$)',
    "q_Glu":    r'q$_{Glu}$ (pmol·cell$^{-1}$·h$^{-1}$)',
    "q_rP":     r'q$_{rP}$ (pg·cell$^{-1}$·h$^{-1}$)',
    "GFP_mean": r'GFP (a.u.)',
    "TMRM_mean":r'TMRM (a.u.)'
}

for x, y in PAIR_CORR:
    if {x, y}.difference(df.columns): continue
    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax  = fig.add_axes(AXES_RECT)
    scatter_by_rep(ax, df, x, y)
    ax.set_xlabel(LABELS.get(x, x))
    ax.set_ylabel(LABELS.get(y, y))
    ax.set_title(f"{x} vs. {y}")
    if x in RATE_SET: ax.set_xlim(left=0)
    if y in RATE_SET: ax.set_ylim(bottom=0)
    safe_legend(ax, title="Clone–Rep", fontsize=8)
    fig.savefig(FIGURE_DIR / "corr" / f"{x}_vs_{y}_raw.png")
    plt.close(fig)

print("✓ Correlations saved")


✓ Time trends saved
✓ Kinetic plots saved
✓ Correlations saved
